In [1]:
import joblib
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from models.managers.db_manager import connect_to_bdd
from sklearn.metrics import accuracy_score

# Connexion à la base de données
conn = connect_to_bdd()

# Requête pour election_2022_t1 et sauvegarde dans df_election
query_election = """
    SELECT *
    FROM election_2022_t1
"""
df_election = pd.read_sql(query_election, conn)

# Requête pour economie et sauvegarde dans df_economie
query_economie = """
    SELECT avg_1982, avg_1983, avg_1984, avg_1985, avg_1986, avg_1987, avg_1988, avg_1989,
           avg_1990, avg_1991, avg_1992, avg_1993, avg_1994, avg_1995, avg_1996, avg_1997,
           avg_1998, avg_1999, avg_2000, avg_2001, avg_2002, avg_2003, avg_2004, avg_2005,
           avg_2006, avg_2007, avg_2008, avg_2009, avg_2010, avg_2011, avg_2012, avg_2013,
           avg_2014, avg_2015, avg_2016, avg_2017, avg_2018, avg_2019, avg_2020, avg_2021,
           avg_2022, avg_2023, code_postal
    FROM economie
"""
df_economie = pd.read_sql(query_economie, conn)

# Requête pour securite et sauvegarde dans df_securite
query_securite = """
    SELECT code_postal, tauxpourcent
    FROM securite
"""
df_securite = pd.read_sql(query_securite, conn)

# Requête pour densite et sauvegarde dans df_densite
query_densite = """
    SELECT code_postal, dens_pop
    FROM demographie
"""
df_densite = pd.read_sql(query_densite, conn)

# Calculer la moyenne de densité de population pour chaque département
df_densite = df_densite.groupby('code_postal')['dens_pop'].mean().reset_index()

# Calculer la moyenne du taux de pourcentage de sécurité pour chaque département
df_securite = df_securite.groupby('code_postal')['tauxpourcent'].mean().reset_index()

# Fusionner les données
df_combined = pd.merge(df_election, df_economie, on="code_postal", how="left")
df_combined = pd.merge(df_combined, df_securite, on="code_postal", how="left")
df_combined = pd.merge(df_combined, df_densite, on="code_postal", how="left")

# Exclure les données de l'Yonne du jeu d'entraînement
df_train = df_combined[df_combined["code_postal"] != '89']

# Sélectionner uniquement les colonnes % Voix/Ins 1 à % Voix/Ins 12
X_train = df_train.loc[:, "% Voix/Ins 1":"% Voix/Ins 12"]
y_train = df_train["winner"]

# Créer le modèle RandomForestClassifier
clf = RandomForestClassifier()

# Entraîner le modèle sur l'ensemble des données d'entraînement
clf.fit(X_train, y_train)

# Requête pour les données de l'Yonne pour les élections
query_election_yonne = """
    SELECT *
    FROM election_2022_t1
    WHERE code_postal = '89'
"""
df_election_yonne = pd.read_sql(query_election_yonne, conn)

# Fermeture de la connexion
conn.close()

# Fusionner les données
df_combined_yonne = pd.merge(df_election_yonne, df_economie, on="code_postal")

# Sélectionner uniquement les colonnes % Voix/Ins 1 à % Voix/Ins 12 pour les données de l'Yonne
X_yonne = df_combined_yonne.loc[:, "% Voix/Ins 1":"% Voix/Ins 12"]
y_yonne = df_combined_yonne["winner"]

# Prédire les probabilités pour chaque classe pour les données de l'Yonne
y_pred_proba = clf.predict_proba(X_yonne)

# Déterminer si chaque parti va dépasser les 5% de votants
parties_depassant_5pct = []
for party_index, party_name in enumerate(clf.classes_):
    proba_threshold = 0.05
    party_proba = y_pred_proba[:, party_index]
    if any(party_proba > proba_threshold):
        parties_depassant_5pct.append((party_name, True))
    else:
        parties_depassant_5pct.append((party_name, False))

# Afficher les résultats
print("Dépassement des 5% pour chaque parti dans l'Yonne :")
for party_name, is_above_5pct in parties_depassant_5pct:
    print(f"Le parti {party_name} dépasse les 5% de votants ? : {is_above_5pct}")


# Calcul de l'accuracy du modèle
accuracy_train = accuracy_score(y_train, clf.predict(X_train))
print("Accuracy sur les données d'entraînement :", accuracy_train)

# Calcul de l'accuracy du modèle sur les données de l'Yonne
accuracy_yonne = accuracy_score(y_yonne, clf.predict(X_yonne))
print("Accuracy sur les données de l'Yonne :", accuracy_yonne)

# Exporter le modèle dans un fichier pickle
path = '../pickle_files/random_forest_model.pkl'
try:
    joblib.dump(clf, path)
    print("Modèle exporté avec succès dans", path)
except Exception as e:
    print("Erreur lors de l'exportation du modèle :", e)

/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_1316/2121615894.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_election = pd.read_sql(query_election, conn)
/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_1316/2121615894.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_economie = pd.read_sql(query_economie, conn)
/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_1316/2121615894.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_securite = pd.read_sql(query_securite, conn)
/

Dépassement des 5% pour chaque parti dans l'Yonne :
Le parti Debout La France dépasse les 5% de votants ? : False
Le parti Europe Écologie Les Verts dépasse les 5% de votants ? : False
Le parti La France Insoumise dépasse les 5% de votants ? : True
Le parti La République En Marche dépasse les 5% de votants ? : True
Le parti Les Républicains dépasse les 5% de votants ? : False
Le parti Lutte Ouvrière dépasse les 5% de votants ? : False
Le parti Parti Communiste Français dépasse les 5% de votants ? : False
Le parti Parti Socialiste dépasse les 5% de votants ? : False
Le parti Rassemblement National dépasse les 5% de votants ? : True
Le parti Reconquête dépasse les 5% de votants ? : False
Le parti Résistons dépasse les 5% de votants ? : False
Accuracy sur les données d'entraînement : 1.0
Accuracy sur les données de l'Yonne : 1.0
Modèle exporté avec succès dans ../pickle_files/random_forest_model.pkl
